In [3]:
import pandas as pd
df1 = pd.read_csv('/content/LOCAL_PEOPLE_DONG_202511.csv')
df2 = pd.read_csv('/content/서울시 상권분석서비스(점포-행정동)_2024년.csv', encoding='euc-kr')
df3 = pd.read_csv('/content/서울시 상권분석서비스(추정매출-행정동)_2024년.csv', encoding='cp949')
df4 = pd.read_csv('/content/소상공인시장진흥공단_상가(상권)정보_서울_202510.csv')

In [4]:
import pandas as pd
import numpy as np

def clean_cols(df):
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip().str.replace("\ufeff", "", regex=False)
    return df

def to_code_str(s):
    return s.astype(str).str.strip()

def to_num(s):
    return pd.to_numeric(s.astype(str).str.replace(",", "", regex=False), errors="coerce")

def make_quarter_from_col(series):
    """
    기준_년분기_코드가 20241, 20242 같은 형태거나 2024Q1 같은 형태인 경우를 최대한 지원.
    결과: '2024Q1' 형태
    """
    def conv(v):
        if pd.isna(v):
            return np.nan
        v = str(v).strip()
        if "Q" in v.upper():
            v2 = v.upper().replace(" ", "")
            # 2024Q1
            return v2
        digits = "".join(ch for ch in v if ch.isdigit())
        if len(digits) >= 5:
            year = digits[:4]
            q = digits[4]
            return f"{year}Q{q}"
        return np.nan
    return series.apply(conv)


In [5]:
df1 = clean_cols(df1)

dong_col = next((c for c in df1.columns if ("행정동" in c and "코드" in c) or c.upper() in ["ADSTRD_CD","H_DNG_CD"]), None)
date_col = next((c for c in df1.columns if "일자" in c or "기준일" in c or c.upper() in ["STD_YMD","STDR_DE"]), None)
hour_col = next((c for c in df1.columns if "시간" in c or c.upper() in ["TMZON","HOUR"]), None)
pop_col  = next((c for c in df1.columns if ("생활" in c and "인구" in c) or ("총" in c and "인구" in c) or c.upper() in ["TOT_LVPOP_CO","TOTAL_POP"]), None)

print("df1 cols picked:", dong_col, date_col, hour_col, pop_col)

living = df1[[dong_col, date_col] + ([hour_col] if hour_col else []) + [pop_col]].copy()
living = living.rename(columns={dong_col:"dong_code", date_col:"date", pop_col:"pop_total", **({hour_col:"hour"} if hour_col else {})})
living["dong_code"] = to_code_str(living["dong_code"])
living["date"] = pd.to_datetime(living["date"], errors="coerce")
living["pop_total"] = to_num(living["pop_total"])
if "hour" in living.columns:
    living["hour"] = to_num(living["hour"])

living["quarter"] = living["date"].dt.to_period("Q").astype(str)  # 예: 2025Q4
living["is_weekend"] = living["date"].dt.dayofweek >= 5
if "hour" in living.columns:
    living["is_night"] = living["hour"].between(18, 23)

g = living.dropna(subset=["dong_code","quarter","pop_total"]).groupby(["dong_code","quarter"], as_index=False)
living_q = g.agg(
    pop_mean=("pop_total","mean"),
    pop_max=("pop_total","max"),
)

weekend_q = (living[living["is_weekend"]]
             .dropna(subset=["dong_code","quarter","pop_total"])
             .groupby(["dong_code","quarter"], as_index=False)
             .agg(pop_weekend_mean=("pop_total","mean")))

living_q = living_q.merge(weekend_q, on=["dong_code","quarter"], how="left")

if "hour" in living.columns:
    night_q = (living[living["is_night"]]
               .dropna(subset=["dong_code","quarter","pop_total"])
               .groupby(["dong_code","quarter"], as_index=False)
               .agg(pop_night_mean=("pop_total","mean")))
    living_q = living_q.merge(night_q, on=["dong_code","quarter"], how="left")

living_q.head()


df1 cols picked: 행정동코드 기준일ID 시간대구분 총생활인구수


,dong_code,quarter,pop_mean,pop_max,pop_weekend_mean,pop_night_mean
0,10000.3822,1970Q1,49.0445,49.0445,NaN,NaN
1,100005.3531,1970Q1,1552.9182,1552.9182,NaN,NaN
2,10001.0481,1970Q1,857.8809,857.8809,NaN,NaN
3,10001.2191,1970Q1,193.2242,193.2242,NaN,NaN
4,10001.3392,1970Q1,363.6429,363.6429,NaN,NaN


In [6]:
df3 = clean_cols(df3)

q_col   = next(c for c in df3.columns if "기준" in c and "년분기" in c)
dong_col= next(c for c in df3.columns if "행정동" in c and "코드" in c and "명" not in c)
biz_col = next(c for c in df3.columns if "서비스" in c and "업종" in c and "코드" in c and "명" not in c)

sales_candidates = [c for c in df3.columns if ("매출" in c and "금액" in c)]
sales_col = sorted(sales_candidates, key=len)[0]

print("df3 cols picked:", q_col, dong_col, biz_col, sales_col)

sales = df3[[q_col, dong_col, biz_col, sales_col]].copy()
sales = sales.rename(columns={q_col:"quarter_raw", dong_col:"dong_code", biz_col:"biz_code", sales_col:"sales_amt"})
sales["dong_code"] = to_code_str(sales["dong_code"])
sales["biz_code"]  = to_code_str(sales["biz_code"])
sales["quarter"]   = make_quarter_from_col(sales["quarter_raw"])
sales["sales_amt"] = to_num(sales["sales_amt"])

sales_q = (sales.dropna(subset=["dong_code","biz_code","quarter"])
                .groupby(["dong_code","quarter","biz_code"], as_index=False)["sales_amt"].sum()
                .sort_values(["dong_code","biz_code","quarter"]))

sales_q["sales_next"] = sales_q.groupby(["dong_code","biz_code"])["sales_amt"].shift(-1)
sales_q["y_growth_rate_nextq"] = (sales_q["sales_next"] / sales_q["sales_amt"]) - 1

sales_q.head()


df3 cols picked: 기준_년분기_코드 행정동_코드 서비스_업종_코드 당월_매출_금액


,dong_code,quarter,biz_code,sales_amt,sales_next,y_growth_rate_nextq
0,11110515,2024Q1,CS100001,3282036149,3.609853e+09,0.099882
33,11110515,2024Q2,CS100001,3609852542,3.010345e+09,-0.166075
65,11110515,2024Q3,CS100001,3010345082,4.315503e+09,0.433557
97,11110515,2024Q4,CS100001,4315502685,NaN,NaN
1,11110515,2024Q1,CS100003,315907116,4.204827e+08,0.331033


In [7]:
df2 = clean_cols(df2)

q_col   = next(c for c in df2.columns if "기준" in c and "년분기" in c)
dong_col= next(c for c in df2.columns if "행정동" in c and "코드" in c and "명" not in c)
biz_col = next(c for c in df2.columns if "서비스" in c and "업종" in c and "코드" in c and "명" not in c)

def pick_col(df, must_have):
    cols = [c for c in df.columns if all(k in c for k in must_have)]
    return cols[0] if cols else None

store_cnt_col = pick_col(df2, ["점포","수"])
open_cnt_col  = pick_col(df2, ["개업","점포"])
close_cnt_col = pick_col(df2, ["폐업","점포"])
fran_cnt_col  = pick_col(df2, ["프랜차이즈","점포"])

print("df2 cols picked:", q_col, dong_col, biz_col, store_cnt_col, open_cnt_col, close_cnt_col, fran_cnt_col)

cols = [q_col, dong_col, biz_col, store_cnt_col, open_cnt_col, close_cnt_col] + ([fran_cnt_col] if fran_cnt_col else [])
store = df2[cols].copy()
store = store.rename(columns={
    q_col:"quarter_raw",
    dong_col:"dong_code",
    biz_col:"biz_code",
    store_cnt_col:"store_cnt",
    open_cnt_col:"open_cnt",
    close_cnt_col:"close_cnt",
    **({fran_cnt_col:"franchise_cnt"} if fran_cnt_col else {})
})

store["dong_code"] = to_code_str(store["dong_code"])
store["biz_code"]  = to_code_str(store["biz_code"])
store["quarter"]   = make_quarter_from_col(store["quarter_raw"])

for c in ["store_cnt","open_cnt","close_cnt","franchise_cnt"]:
    if c in store.columns:
        store[c] = to_num(store[c]).fillna(0)

agg_cols = ["store_cnt","open_cnt","close_cnt"] + (["franchise_cnt"] if "franchise_cnt" in store.columns else [])
store_q = (store.dropna(subset=["dong_code","biz_code","quarter"])
                .groupby(["dong_code","quarter","biz_code"], as_index=False)[agg_cols].sum()
                .sort_values(["dong_code","biz_code","quarter"]))

store_q["close_rate"] = np.where(store_q["store_cnt"]>0, store_q["close_cnt"]/store_q["store_cnt"], np.nan)
store_q["net_open_rate"] = np.where(store_q["store_cnt"]>0, (store_q["open_cnt"]-store_q["close_cnt"])/store_q["store_cnt"], np.nan)
if "franchise_cnt" in store_q.columns:
    store_q["franchise_ratio"] = np.where(store_q["store_cnt"]>0, store_q["franchise_cnt"]/store_q["store_cnt"], np.nan)

store_q["y_risk_close_rate_nextq"] = store_q.groupby(["dong_code","biz_code"])["close_rate"].shift(-1)

store_q.head()


df2 cols picked: 기준_년분기_코드 행정동_코드 서비스_업종_코드 점포_수 개업_점포_수 폐업_점포_수 프랜차이즈_점포_수


,dong_code,quarter,biz_code,store_cnt,open_cnt,close_cnt,franchise_cnt,close_rate,net_open_rate,franchise_ratio,y_risk_close_rate_nextq
0,11110515,2024Q1,CS100001,73,2,2,0,0.027397,0.000000,0.0,0.000000
78,11110515,2024Q2,CS100001,77,4,0,0,0.000000,0.051948,0.0,0.053333
156,11110515,2024Q3,CS100001,75,2,4,0,0.053333,-0.026667,0.0,0.012821
234,11110515,2024Q4,CS100001,78,4,1,0,0.012821,0.038462,0.0,NaN
1,11110515,2024Q1,CS100002,2,0,0,0,0.000000,0.000000,0.0,0.000000


In [8]:
df4 = clean_cols(df4)

dong_candidates = ["행정동코드","ADSTRD_CD","adm_cd","ADM_CD"]
bjd_candidates  = ["법정동코드","BJDONG_CD","bjd_cd"]

dong_col = next((c for c in dong_candidates if c in df4.columns), None)
bjd_col  = next((c for c in bjd_candidates if c in df4.columns), None)

small_candidates = [c for c in df4.columns if ("소분류" in c and "코드" in c)] + \
                  [c for c in df4.columns if ("상권업종" in c and "소" in c and "코드" in c)]
small_col = small_candidates[0] if small_candidates else None

print("df4 cols picked:", dong_col, bjd_col, small_col)

if dong_col and small_col:
    sbiz = df4[[dong_col, small_col]].copy()
    sbiz = sbiz.rename(columns={dong_col:"dong_code", small_col:"sbiz_small"})
    sbiz["dong_code"] = to_code_str(sbiz["dong_code"])
    sbiz["sbiz_small"] = to_code_str(sbiz["sbiz_small"])

    sbiz_cnt = (sbiz.dropna()
                    .groupby(["dong_code","sbiz_small"], as_index=False)
                    .size()
                    .rename(columns={"size":"shop_cnt"}))
    sbiz_cnt.head()
else:
    print("⚠️ df4에 행정동코드가 없거나 세분류 업종코드가 없어 바로 집계가 어려워요.")
    print("   - 법정동코드만 있으면 법정동 단위로 먼저 집계한 뒤, 법정동→행정동 매핑 또는 공간조인이 필요합니다.")


df4 cols picked: 행정동코드 법정동코드 상권업종소분류코드


In [9]:
panel = sales_q.merge(
    store_q,
    on=["dong_code","quarter","biz_code"],
    how="inner"
)

panel = panel.merge(living_q, on=["dong_code","quarter"], how="left")

panel_model = panel.dropna(subset=["y_growth_rate_nextq","y_risk_close_rate_nextq"]).copy()

panel_model.head()


,dong_code,quarter,biz_code,sales_amt,sales_next,y_growth_rate_nextq,store_cnt,open_cnt,close_cnt,franchise_cnt,close_rate,net_open_rate,franchise_ratio,y_risk_close_rate_nextq,pop_mean,pop_max,pop_weekend_mean,pop_night_mean
0,11110515,2024Q1,CS100001,3282036149,3.609853e+09,0.099882,73,2,2,0,0.027397,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
1,11110515,2024Q2,CS100001,3609852542,3.010345e+09,-0.166075,77,4,0,0,0.000000,0.051948,0.000000,0.053333,NaN,NaN,NaN,NaN
2,11110515,2024Q3,CS100001,3010345082,4.315503e+09,0.433557,75,2,4,0,0.053333,-0.026667,0.000000,0.012821,NaN,NaN,NaN,NaN
4,11110515,2024Q1,CS100003,315907116,4.204827e+08,0.331033,18,2,2,1,0.111111,0.000000,0.055556,0.058824,NaN,NaN,NaN,NaN
5,11110515,2024Q2,CS100003,420482695,3.914130e+08,-0.069134,17,0,1,1,0.058824,-0.058824,0.058824,0.125000,NaN,NaN,NaN,NaN


In [10]:
print(panel_model.shape)
print(panel_model.columns)

print(panel_model["y_growth_rate_nextq"].describe())
print(panel_model["y_risk_close_rate_nextq"].describe())


(50258, 18)
Index(['dong_code', 'quarter', 'biz_code', 'sales_amt', 'sales_next',
       'y_growth_rate_nextq', 'store_cnt', 'open_cnt', 'close_cnt',
       'franchise_cnt', 'close_rate', 'net_open_rate', 'franchise_ratio',
       'y_risk_close_rate_nextq', 'pop_mean', 'pop_max', 'pop_weekend_mean',
       'pop_night_mean'],
      dtype='object')
count    50258.000000
mean         1.319626
std        170.756064
min         -0.999950
25%         -0.133734
50%          0.000000
75%          0.149113
max      37999.000000
Name: y_growth_rate_nextq, dtype: float64
count    50258.000000
mean         0.038555
std          0.098520
min          0.000000
25%          0.000000
50%          0.000000
75%          0.043478
max          4.000000
Name: y_risk_close_rate_nextq, dtype: float64


In [11]:
panel_model.sort_values("y_growth_rate_nextq", ascending=False).head(10)[
    ["dong_code","quarter","biz_code","sales_amt","sales_next","y_growth_rate_nextq"]
]

,dong_code,quarter,biz_code,sales_amt,sales_next,y_growth_rate_nextq
66802,11740620,2024Q2,CS200032,630,2.394000e+07,37999.000000
33363,11440660,2024Q2,CS300031,671914,1.911223e+09,2843.445800
58941,11680630,2024Q1,CS300019,1300969,2.215505e+09,1701.965615
20464,11305555,2024Q1,CS300031,577,9.225910e+05,1597.944541
34927,11470520,2024Q1,CS300035,102644,1.468840e+08,1430.004482
14727,11260570,2024Q3,CS200026,1216,1.604693e+06,1318.648849
7224,11200560,2024Q1,CS300026,7135,6.000000e+06,839.925018
38438,11500540,2024Q2,CS300024,111250,8.241485e+07,739.807604
56872,11650652,2024Q2,CS300027,968742,6.892016e+08,710.439755
63601,11710641,2024Q2,CS200017,48080,3.365632e+07,699.006718


In [12]:
panel_model.sort_values("y_risk_close_rate_nextq", ascending=False).head(10)[
    ["dong_code","quarter","biz_code","store_cnt","close_cnt","y_risk_close_rate_nextq"]
]

,dong_code,quarter,biz_code,store_cnt,close_cnt,y_risk_close_rate_nextq
63329,11710631,2024Q2,CS300002,1,0,4.0
38557,11500550,2024Q2,CS300002,2,1,3.0
56324,11650621,2024Q3,CS100007,2,1,2.0
61355,11710532,2024Q1,CS300029,5,0,2.0
48466,11590520,2024Q1,CS300002,1,0,2.0
50465,11590680,2024Q2,CS100007,3,1,2.0
35797,11470570,2024Q1,CS300002,1,1,2.0
60389,11680730,2024Q2,CS100006,1,0,2.0
30604,11410620,2024Q2,CS300002,2,2,2.0
23276,11320670,2024Q1,CS300002,2,0,2.0


In [13]:
df = panel_model.copy()

df = df[(df["sales_amt"] >= 100000) & (df["store_cnt"] >= 5)].copy()

df["y_growth_cls"] = (
    df.groupby("quarter")["y_growth_rate_nextq"]
      .transform(lambda s: (s >= s.quantile(0.8)).astype(int))
)

df["y_risk_cls"] = (
    df.groupby("quarter")["y_risk_close_rate_nextq"]
      .transform(lambda s: (s >= s.quantile(0.8)).astype(int))
)

df[["y_growth_rate_nextq","y_risk_close_rate_nextq","y_growth_cls","y_risk_cls"]].describe()

,y_growth_rate_nextq,y_risk_close_rate_nextq,y_growth_cls,y_risk_cls
count,43624.000000,43624.000000,43624.000000,43624.000000
mean,0.438863,0.035536,0.200028,0.203122
std,18.388556,0.070110,0.400025,0.402327
min,-0.999950,0.000000,0.000000,0.000000
25%,-0.128231,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.145402,0.049180,0.000000,0.000000
max,2843.445800,2.000000,1.000000,1.000000


In [14]:
print(df["y_growth_cls"].value_counts(normalize=True))
print(df["y_risk_cls"].value_counts(normalize=True))

y_growth_cls
0    0.799972
1    0.200028
Name: proportion, dtype: float64
y_risk_cls
0    0.796878
1    0.203122
Name: proportion, dtype: float64


In [15]:
df = df[(df["sales_amt"] >= 100000) & (df["store_cnt"] >= 5)].copy()

In [16]:
df.to_parquet("features_panel.parquet", index=False)

In [17]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs("/content/drive/MyDrive/상권분석", exist_ok=True)

out_path = "/content/drive/MyDrive/상권분석/features_panel.parquet"
df.to_parquet(out_path, index=False)

print("saved:", out_path)


Mounted at /content/drive
saved: /content/drive/MyDrive/상권분석/features_panel.parquet


In [18]:
df.to_parquet("/content/features_panel.parquet", index=False)
from google.colab import files
files.download("/content/features_panel.parquet")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
biz_name_col = None
src = None

for cand in ["서비스_업종_코드_명", "서비스업종코드명", "서비스_업종_코드_명 "]:
    if cand in df2.columns:
        biz_name_col = cand
        src = df2
        break
    if cand in df3.columns:
        biz_name_col = cand
        src = df3
        break

if src is not None and biz_name_col is not None:
    mapping = (src[["서비스_업종_코드", biz_name_col]]
               .dropna()
               .drop_duplicates()
               .rename(columns={"서비스_업종_코드":"biz_code", biz_name_col:"biz_name"}))

    mapping["biz_code"] = mapping["biz_code"].astype(str).str.strip()
    mapping["biz_name"] = mapping["biz_name"].astype(str).str.strip()

    mapping.to_csv("/content/biz_code_map.csv", index=False, encoding="utf-8-sig")
    print("saved biz_code_map.csv rows:", len(mapping))
    mapping.head()
else:
    print("업종명 컬럼을 못 찾았어요. df2/df3 컬럼명을 한번 확인해 주세요.")


saved biz_code_map.csv rows: 100


In [21]:
# 참조 데이터
# 행정 코드 -> 행정 동
import os, zipfile, tempfile
from pathlib import Path
import pandas as pd
import geopandas as gpd

ZIP_PATH = Path("서울시 상권분석서비스(영역-행정동).zip")
OUT_CSV = Path("dong_map.csv")

def find_file(root: str, ext: str):
    for r, _, files in os.walk(root):
        for f in files:
            if f.lower().endswith(ext.lower()):
                return os.path.join(r, f)
    return None

tmpdir = tempfile.mkdtemp(prefix="dong_")
with zipfile.ZipFile(ZIP_PATH) as z:
    z.extractall(tmpdir)

shp_path = find_file(tmpdir, ".shp")
cpg_path = find_file(tmpdir, ".cpg")

if shp_path is None:
    raise FileNotFoundError("압축 해제 후 .shp 파일을 못 찾았어요.")

encoding = "utf-8"
if cpg_path:
    try:
        encoding = Path(cpg_path).read_text(encoding="utf-8").strip() or "utf-8"
    except Exception:
        encoding = "utf-8"

df = gpd.read_file(shp_path, encoding=encoding, ignore_geometry=True)

dong_map = df[["ADSTRD_CD","ADSTRD_NM"]].copy()
dong_map.columns = ["dong_code","dong_name"]
dong_map["dong_code"] = dong_map["dong_code"].astype(str).str.strip()
dong_map["dong_name"] = dong_map["dong_name"].astype(str).str.strip()
dong_map = dong_map.drop_duplicates().sort_values("dong_code")

dong_map.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("saved ->", OUT_CSV.resolve(), "rows:", len(dong_map))
dong_map.head()


saved -> /content/dong_map.csv rows: 425


,dong_code,dong_name
0,11110515,청운효자동
1,11110530,사직동
2,11110540,삼청동
3,11110550,부암동
4,11110560,평창동
